In [15]:
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten
from keras.layers import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
import keras.optimizers as Optimizer
import matplotlib.pyplot as plt
import numpy as np

In [16]:
# Defining input image dimension

img_row = 28
img_col = 28
channel = 1
img_shape = (img_row, img_col, channel)

In [17]:
# Defining Generator class

def build_generator():
  noise_shape = (100,)  #this is the size of the latent vector

  model = Sequential()
  model.add(Dense(256, input_shape = noise_shape))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum = 0.8))
  model.add(Dense(512))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum = 0.8))
  model.add(Dense(1024))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum =  0.8))

  model.add(Dense(np.prod(img_shape), activation = 'tanh'))
  model.add(Reshape(img_shape))
  model.summary()

  noise = Input(shape= noise_shape)
  fake_img = model(noise)    #Generated Image
  return Model(noise, fake_img)

In [18]:
# Building Discriminator Class

def build_discriminator():

  model = Sequential()
  model.add(Flatten(input_shape = img_shape))
  model.add(Dense(512))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dense(256))
  model.add(LeakyReLU(alpha =0.2))
  model.add(Dense(1, activation = 'sigmoid'))
  model.summary()

  img = Input(shape=img_shape)
  validity = model(img)   # 0 for Generated image and 1 for Real image

  return Model(img, validity)


In [19]:
# Training the models

def train(epochs, batch_size=128, save_interval=500):
  
  #Load the dataset
  (xtrain,_),(_,_) = mnist.load_data()
  xtrain = (xtrain.astype(np.float32) - 127.5) / 127.5
  xtrain = np.expand_dims(xtrain, axis = 3)
  half_batch = int(batch_size/2)

  for epoch in range(epochs):

    # Training the Discriminator

    idx = np.random.randint(0, xtrain.shape[0], half_batch)
    real_imgs = xtrain[idx]    # taking random samples from the training set to pass to the dicriminator

    # Generating a half batch of fake images
    noise = np.random.normal(0, 1, (half_batch, 100))
    gen_imgs = generator.predict(noise)

    d_loss_real = discriminator.train_on_batch(real_imgs, np.ones((half_batch,1)))
    d_loss_gen = discriminator.train_on_batch(gen_imgs, np.zeros((half_batch,1)))
    d_loss = 0.5 * (np.add(d_loss_real, d_loss_gen))

    # Training the Generator

    noise = np.random.normal(0,1, (batch_size,100))
    valid_y = np.array([1] * batch_size)
    g_loss = combined.train_on_batch(noise, valid_y)

    print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

    if epoch % save_interval == 0:
      save_imgs(epoch)

In [20]:
# Function to save the images
def save_imgs(epoch):
  r, c = 5, 5
  noise = np.random.normal(0, 1, (r * c, 100))
  gen_imgs = generator.predict(noise)

  gen_imgs = 0.5 * gen_imgs + 0.5

  fig, axs = plt.subplots(r,c)
  cnt = 0

  for i in range(r):
    for j in range(c):
      axs[i,j].imshow(gen_imgs[cnt, :, :, 0], cmap = 'gray')
      axs[i,j].axis('off')
      cnt += 1
  fig.savefig("C:\\Users\\hp\\Desktop\\Images\\mnist_%d.png" %epoch)
  plt.close()


In [21]:
import tensorflow as tf

optimizer = tf.keras.optimizers.Adam(0.0002, 0.5)

In [22]:
discriminator = build_discriminator()
discriminator.compile(loss = 'binary_crossentropy', optimizer = optimizer, 
                     metrics = ['accuracy'])
generator = build_generator()
generator.compile(loss = 'binary_crossentropy', optimizer = optimizer) 

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 784)               0         
                                                                 
 dense_14 (Dense)            (None, 512)               401920    
                                                                 
 leaky_re_lu_10 (LeakyReLU)  (None, 512)               0         
                                                                 
 dense_15 (Dense)            (None, 256)               131328    
                                                                 
 leaky_re_lu_11 (LeakyReLU)  (None, 256)               0         
                                                                 
 dense_16 (Dense)            (None, 1)                 257       
                                                                 
Total params: 533,505
Trainable params: 533,505
Non-tr

In [23]:
# Defining the input noise
z = Input(shape = (100,))
fake_img = generator(z)
discriminator.trainable = False
valid = discriminator(fake_img)

In [24]:
# Defining the combined model

combined = Model(z,valid)
combined.compile(loss='binary_crossentropy', optimizer = optimizer)

In [26]:
train(epochs = 20000, batch_size = 32, save_interval = 500)

Streaming output truncated to the last 5000 lines.
15000 [D loss: 0.719397, acc.: 62.50%] [G loss: 0.884038]
15001 [D loss: 0.739926, acc.: 46.88%] [G loss: 0.861217]
15002 [D loss: 0.751789, acc.: 50.00%] [G loss: 0.854281]
15003 [D loss: 0.689814, acc.: 62.50%] [G loss: 0.845780]
15004 [D loss: 0.649288, acc.: 65.62%] [G loss: 0.927503]
15005 [D loss: 0.671861, acc.: 56.25%] [G loss: 0.932641]
15006 [D loss: 0.682009, acc.: 46.88%] [G loss: 0.958746]
15007 [D loss: 0.663716, acc.: 53.12%] [G loss: 0.888794]
15008 [D loss: 0.677073, acc.: 59.38%] [G loss: 0.861164]
15009 [D loss: 0.652844, acc.: 56.25%] [G loss: 0.843848]
15010 [D loss: 0.655348, acc.: 59.38%] [G loss: 0.885931]
15011 [D loss: 0.658655, acc.: 59.38%] [G loss: 0.857086]
15012 [D loss: 0.688566, acc.: 50.00%] [G loss: 0.896562]
15013 [D loss: 0.691562, acc.: 62.50%] [G loss: 0.875209]
15014 [D loss: 0.652217, acc.: 65.62%] [G loss: 0.849699]
15015 [D loss: 0.737556, acc.: 50.00%] [G loss: 0.891347]
15016 [D loss: 0.6954